## Case 3: Процесс электронно-лучевой сварки
Выпускной проект по курсу: Аналитик данных МГТУ им. Н.Э. Баумана 

Группа: AD11902/1

Слушатель: Русаков Глеб Игоревич

Задача: определить значение глубины и ширины сварочного шва электронно-лучевой сварки.\
Регрессия.

Целевые переменные: 

* Depth - глубина шва
* Width - ширина шва

Используемые признаки: 

* IW - величина сварочного тока
* IF - ток фокусировки электронного пучка
* VW - скорость сварки
* FP - расстояние от поверхности образцов до электронно-оптической системы 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import stats

Загрузка датасета

In [ ]:
data_orig = pd.read_csv('ebw_data.csv')

Анализ данных

In [ ]:
data_orig[['IW', 'IF', 'VW', 'FP']].plot()
plt.legend(loc='upper right',framealpha=0.3)

In [ ]:
data_orig.head()

Проверка пропусков

In [ ]:
data_orig.isnull().sum()

In [ ]:
data_orig.info()

Описательная статистика

In [ ]:
data_orig.describe().T

Поиск дубликатов

In [ ]:
data_orig.nunique()

In [ ]:
data_orig.duplicated().sum()

In [ ]:
duplicate_data = data_orig[data_orig.duplicated()]
duplicate_data

Проверка пропусков

In [ ]:
data_orig.isnull().sum()

Удаление дубликатов

In [ ]:
data_orig_deduplicated = data_orig.drop_duplicates()

Анализ после удаления дубликатов

In [ ]:
data_orig_deduplicated.info()

In [ ]:
data_orig_deduplicated[['IW', 'IF', 'VW', 'FP']].plot()
plt.legend(loc='upper right', framealpha=0.3)

In [ ]:
data_orig_deduplicated.describe().T

Убедимся, что все дубликаты удалены

In [ ]:
data_orig_deduplicated.duplicated().sum()

Попарная зависимость переменных и гистограммы

In [ ]:
sns.pairplot(data_orig_deduplicated)

Корреляция данных

In [ ]:
data_orig_deduplicated.corr().round(2)

In [ ]:
dataplot = sns.heatmap(data_orig_deduplicated.corr(), cmap='YlGnBu', annot = True)
plt.rcParams['figure.figsize'] = (15, 15)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data_orig_deduplicated)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
test = sns.boxplot(data_orig_deduplicated['VW'])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data_orig_deduplicated['FP'])
plt.show()

In [ ]:
Q1VW = np.quantile(data_orig_deduplicated['VW'], 0.25)
Q3VW = np.quantile(data_orig_deduplicated['VW'], 0.75)
IQRVW = Q3VW - Q1VW
lowerVW = Q1VW - 1.5 * IQRVW
upperVW = Q3VW + 1.5 * IQRVW

Q1FP = np.quantile(data_orig_deduplicated['FP'], 0.25)
Q3FP = np.quantile(data_orig_deduplicated['FP'], 0.75)
IQRFP = Q3FP - Q1FP
lowerFP = Q1FP - 1.5 * IQRFP
upperFP = Q3FP + 1.5 * IQRFP

In [ ]:
print(lowerVW, upperVW)

In [ ]:
print(lowerFP, upperFP)

In [ ]:
QVW = ((data_orig_deduplicated['VW'] > lowerVW) & (data_orig_deduplicated['VW'] < upperVW))
QFP = ((data_orig_deduplicated['FP'] > lowerFP) & (data_orig_deduplicated['FP'] < upperFP))

In [ ]:
data_orig_drop = data_orig_deduplicated[QVW & QFP]

In [ ]:
data_orig_drop.isna().sum()

In [ ]:
data_orig_drop.info()

In [ ]:
data_orig_drop['VW'].value_counts()

In [ ]:
data_orig_drop.reset_index(inplace=True)

In [ ]:
data_orig_drop.info()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data_orig_drop[['IW', 'IF', 'VW', 'FP', 'Depth', 'Width']])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
test = sns.boxplot(data_orig_drop['VW'])
plt.show()

In [ ]:
Q1VW = np.quantile(data_orig_drop['VW'], 0.25)
Q3VW = np.quantile(data_orig_drop['VW'], 0.75)
IQRVW = Q3VW - Q1VW
lowerVW = Q1VW - 1.5 * IQRVW
upperVW = Q3VW + 1.5 * IQRVW

In [ ]:
print(lowerVW, upperVW)

In [ ]:
QVW = ((data_orig_drop['VW'] > lowerVW) & (data_orig_drop['VW'] < upperVW))

In [ ]:
data_orig_drop = data_orig_drop[QVW]

In [ ]:
data_orig_drop.reset_index(inplace=True)

In [ ]:
data_orig_drop.info()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data_orig_drop[['IW', 'IF', 'VW', 'FP', 'Depth', 'Width']])
plt.show()

Анализ данных без выбросов

In [ ]:
data_orig_drop.drop(columns=['index', 'level_0'], axis=1, inplace=True)

In [ ]:
data_orig_drop[['IW', 'IF', 'VW', 'FP']].plot()
plt.legend(loc='upper right', framealpha=0.3)

In [ ]:
data_orig_drop.describe().T

In [ ]:
sns.pairplot(data_orig_drop)

Оценка плотности ядра

In [ ]:
_, ax = plt.subplots(figsize=(15, 12))
data_orig_drop[['IW', 'IF', 'VW', 'FP']].plot(kind='kde', ax=ax)

Построение и обучение моделей. Нормализация

In [ ]:
x_columns = ['IW', 'IF', 'VW', 'FP']
y_columns = ['Depth', 'Width']
# X = pd.DataFrame(data_orig_drop, columns = x_columns)
# y = pd.DataFrame(data_orig_drop, columns = y_columns)

In [ ]:
X = pd.DataFrame(data_orig_drop, columns = x_columns)
y = pd.DataFrame(data_orig_drop, columns = y_columns)

In [ ]:
X.head()

In [ ]:
y.head()

Нормализация

In [ ]:
minmax_scaler = MinMaxScaler()
# data_x_norm = minmax_scaler.fit_transform(np.array(X[['IW', 'IF', 'VW', 'FP']]))
# data_norm_x_df = pd.DataFrame(data = data_x_norm, columns = ['IW', 'IF', 'VW', 'FP'])

In [ ]:
data_x_norm = minmax_scaler.fit_transform(np.array(X[['IW', 'IF', 'VW', 'FP']]))

In [ ]:
data_norm_x_df = pd.DataFrame(data = data_x_norm, columns = ['IW', 'IF', 'VW', 'FP'])

In [ ]:
data_norm_x_df.head()

In [ ]:
data_norm_x_df.info()

In [ ]:
data_norm_x_df.describe().T

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data_norm_x_df)
plt.show()

In [ ]:
print(data_norm_x_df['IW'].unique())
print(data_norm_x_df['IF'].unique())
print(data_norm_x_df['VW'].unique())
print(data_norm_x_df['FP'].unique())

In [ ]:
_, ax = plt.subplots(figsize=(15, 12))
data_norm_x_df[['IW', 'IF', 'VW', 'FP']].plot(kind='kde', ax=ax)

In [ ]:
# x_columns = ['IW', 'IF', 'VW', 'FP']
# y_columns = ['Depth', 'Width']

In [ ]:
# X = pd.DataFrame(data_norm_df, columns = x_columns)
# y = pd.DataFrame(data_norm_df, columns= y_columns)

In [ ]:
# X.head()

In [ ]:
# y.head()

Разделение датасета на обучающую и тренировочную выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_norm_x_df, y, train_size = 0.8, random_state=42)

Проверка размерностей

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

Построение нейросети

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

Входной размер

In [ ]:
n_features = data_norm_x_df.shape[1]

Построение модели и обучение

In [91]:


model = Sequential()

model.add(Dense(64, activation = 'relu', input_shape = (n_features,)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(2, activation = 'relu'))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics='accuracy')

history = model.fit(X_train, y_train,
                    epochs=150,
                    batch_size=7,
                    verbose=1
                    )
print(history.history['accuracy'])

Epoch 1/150
5/5 [==============================] - 2s 2ms/step - loss: 2.3675 - accuracy: 0.0000e+00
Epoch 2/150
5/5 [==============================] - 0s 3ms/step - loss: 2.3361 - accuracy: 0.0000e+00
Epoch 3/150
5/5 [==============================] - 0s 3ms/step - loss: 2.1736 - accuracy: 0.0000e+00
Epoch 4/150
5/5 [==============================] - 0s 4ms/step - loss: 1.9710 - accuracy: 0.0000e+00
Epoch 5/150
5/5 [==============================] - 0s 4ms/step - loss: 1.6424 - accuracy: 0.0000e+00
Epoch 6/150
5/5 [==============================] - 0s 2ms/step - loss: 1.2693 - accuracy: 0.0000e+00
Epoch 7/150
5/5 [==============================] - 0s 3ms/step - loss: 0.8661 - accuracy: 0.0000e+00
Epoch 8/150
5/5 [==============================] - 0s 3ms/step - loss: 0.4750 - accuracy: 0.1000
Epoch 9/150
5/5 [==============================] - 0s 3ms/step - loss: 0.1843 - accuracy: 1.0000
Epoch 10/150
5/5 [==============================] - 0s 4ms/step - loss: 0.1201 - accuracy: 1.0000
E

Визуализация потери и точности

In [ ]:
model.evaluate(X_test, y_test)

Без удаления выбросов

In [ ]:
x_columns = ['IW', 'IF', 'VW', 'FP']
y_columns = ['Depth', 'Width']
X2 = pd.DataFrame(data_orig_deduplicated, columns = x_columns)
y2 = pd.DataFrame(data_orig_deduplicated, columns = y_columns)

In [ ]:
minmax_scaler = MinMaxScaler()
data_x_norm2 = minmax_scaler.fit_transform(np.array(X2[['IW', 'IF', 'VW', 'FP']]))
data_norm_x_df2 = pd.DataFrame(data = data_x_norm2, columns = ['IW', 'IF', 'VW', 'FP'])

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(data_norm_x_df2, y2, train_size = 0.7, random_state=42)


In [ ]:
n_features2 = X_train2.shape[1]

In [ ]:


modelnn = Sequential()

modelnn.add(Dense(128, activation = 'relu', input_shape = (n_features2,)))
modelnn.add(Dense(128, activation = 'relu'))
modelnn.add(Dense(128, activation = 'relu'))
modelnn.add(Dense(64, activation = 'relu'))
modelnn.add(Dense(32, activation = 'relu'))
modelnn.add(Dense(16, activation = 'relu'))
modelnn.add(Dense(2, activation = 'relu'))

modelnn.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics='accuracy')

historynn = modelnn.fit(X_train2, y_train2,
                    epochs=25,
                    batch_size=3,
                    verbose=1,
                    validation_split = 0.2
                    )
print(historynn.history['accuracy'])

In [ ]:
prednn = modelnn.predict(X_train2)

In [ ]:
y_train2

In [ ]:
prednn

In [ ]:
modelnn.evaluate(X_test2)

In [ ]:
y_train

In [ ]:
pred = model.predict(X_train)

In [ ]:
pred

In [ ]:
y_train

In [ ]:
test = model.evaluate(X_test2, y_test2, batch_size=3, verbose=1)
test

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model2 = LinearRegression()

model2.fit(X_train2, y_train2)

In [ ]:
model2.score(X_test2, y_test2)

In [ ]:
y_pred = model2.predict(X_test2)

print(y_pred[:3])

In [ ]:
from sklearn import metrics

print('Среднеквадратическая ошибка (RMSE):', np.sqrt(metrics.mean_squared_error(y_test2, y_pred)).round(2))

In [ ]:
print('R2:', np.round(metrics.r2_score(y_test2, y_pred), 2))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model3 = DecisionTreeRegressor()
model3.fit(X_train2, y_train2)

In [ ]:
model3.score(X_test2, y_test2)

In [ ]:
y_pred2 = model3.predict(X_test2)
print(y_pred[:3])

In [ ]:
print('Среднеквадратическая ошибка (RMSE):', np.sqrt(metrics.mean_squared_error(y_test2, y_pred2)).round(2))

In [ ]:
print('R2:', np.round(metrics.r2_score(y_test2,y_pred2), 2))

In [ ]:
from sklearn.linear_model import MultiTaskLassoCV

In [ ]:

model4 = MultiTaskLassoCV()
model4.fit(X_train2, y_train2)

In [ ]:
model4.score(X_test2, y_test2)

In [ ]:
y_pred3 = model4.predict(X_test2)

In [ ]:
print('Среднеквадратическая ошибка (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred3)).round(2))

In [ ]:
print('R2:', np.round(metrics.r2_score(y_test2,y_pred3), 2))